In [ ]:
# !pip install underthesea textblob spellchecker nltk

     ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
     -- ------------------------------------- 0.3/3.9 MB ? eta -:--:--
     ------------------ --------------------- 1.8/3.9 MB 7.2 MB/s eta 0:00:01
     ----------------------------- ---------- 2.9/3.9 MB 5.4 MB/s eta 0:00:01
     ---------------------------------- ----- 3.4/3.9 MB 4.8 MB/s eta 0:00:01
     ---------------------------------------- 3.9/3.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/20.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/20.9 MB 2.8 MB/s eta

In [4]:
import pandas as pd
import re
from langdetect import detect
from underthesea import text_normalize
from textblob import TextBlob
from spellchecker import SpellChecker
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
# Khởi tạo spellchecker cho tiếng Việt
spell_vi = SpellChecker(language=None)
spell_vi.word_frequency.load_words(["công nghệ", "thông tin", "học máy", "MOR"])

In [7]:
# 📌 1. Xác định ngôn ngữ
def detect_language(text):
    try:
        lang = detect(text)
        return lang if lang in ["en", "vi"] else "unknown"
    except:
        return "unknown"

In [8]:
# 📌 2. Làm sạch dữ liệu chung
def clean_text(text):
    text = text.strip()  # Xóa khoảng trắng đầu/cuối
    text = re.sub(r'\s+', ' ', text)  # Chuẩn hóa khoảng trắng
    text = re.sub(r'[^\w\s,.!?-]', '', text)  # Loại bỏ ký tự đặc biệt không mong muốn
    return text

In [9]:
# 📌 3. Chuẩn hóa tiếng Việt
def normalize_vietnamese(text):
    return text_normalize(text)

def correct_spelling_vietnamese(text):
    words = text.split()
    corrected_text = " ".join([spell_vi.correction(word) or word for word in words])
    return corrected_text

In [10]:
# 📌 4. Chuẩn hóa tiếng Anh
def normalize_english(text):
    text = text.lower()
    return text

def correct_english_spelling(text):
    return str(TextBlob(text).correct())

In [11]:
# 📌 5. Tiền xử lý văn bản chung
def preprocess_text(text):
    lang = detect_language(text)
    text = clean_text(text)
    
    if lang == "vi":
        text = normalize_vietnamese(text)
        text = correct_spelling_vietnamese(text)
    elif lang == "en":
        text = normalize_english(text)
        text = correct_english_spelling(text)

    return text, lang


In [12]:
# 📌 6. Đọc file CSV và chuẩn hóa dữ liệu
def process_and_save_data(csv_file, output_file="processed_data.csv"):
    df = pd.read_csv(csv_file)
    
    if "Question" not in df.columns or "Answer" not in df.columns:
        print("⚠️ Lỗi: File CSV phải có cột 'Question' và 'Answer'.")
        return
    
    processed_data = []

    for _, row in df.iterrows():
        question, q_lang = preprocess_text(str(row["Question"]))
        answer, a_lang = preprocess_text(str(row["Answer"]))
        
        processed_data.append([question, q_lang, answer, a_lang])
    
    processed_df = pd.DataFrame(processed_data, columns=["Question", "Question_Language", "Answer", "Answer_Language"])
    processed_df.to_csv(output_file, index=False, encoding="utf-8-sig")
    
    print(f"✅ Dữ liệu đã được chuẩn hóa và lưu vào {output_file}")

In [13]:
# 📌 7. Chạy chương trình
if __name__ == "__main__":
    process_and_save_data("./database/database.csv")

✅ Dữ liệu đã được chuẩn hóa và lưu vào processed_data.csv
